In [ ]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd /kaggle/working/segment-anything-2
%pip install -e .
%cd /kaggle/working/segment-anything-2/checkpoints/
!bash /kaggle/working/segment-anything-2/checkpoints/download_ckpts.sh
%cd /kaggle/working/segment-anything-2/

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 134.70 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (376/376), done.
/kaggle/working/segment-anything-2
Obtaining file:///kaggle/working/segment-anything-2


In [ ]:
!wget https://storage.yandexcloud.net/ds-ods/files/files/2ad601fd/test.zip
!wget https://storage.yandexcloud.net/ds-ods/files/submissions/21dd71fc-63e7-4419-8cc6-ec6af1214dee/62ab2c57/NTO_SUB0%202.zip
!pip install supervision 

In [ ]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
%capture
!unzip /kaggle/working/segment-anything-2/test.zip

In [ ]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [ ]:
!wget https://huggingface.co/datasets/andrey200702/REC_SYS/resolve/main/sm_base_6ep.pth?download=true
!mv ./sm_base_6ep.pth?download=true ./sm_base_6ep.pth

In [ ]:
!wget https://huggingface.co/datasets/andrey200702/REC_SYS/resolve/main/sm_base_3ep.pth?download=true
!mv ./sm_base_3ep.pth?download=true ./sm_base_3ep.pth

In [ ]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "./sm_base_6ep.pth"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model_large = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor_large = SAM2ImagePredictor(sam2_model_large)

In [ ]:
%capture
!pip install ultralytics supervision

In [ ]:
from  ultralytics import YOLO
import supervision as sv
model = YOLO('/kaggle/input/best-yolo/best (1).pt')

def predict_yolo_image(image):
    return sv.Detections.from_ultralytics(model(image,conf=0.4)[0])

In [ ]:
def predict_sam(image,box):
    predictor.set_image(image)
    masks, scores, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=box,
        multimask_output=False,
    )
    return masks, scores

In [ ]:
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [ ]:
from tqdm.auto import tqdm
import supervision as sv
import pycocotools
import json
import pycocotools.mask
import numpy as np
from ultralytics import YOLO
import albumentations as A
import os

test_folder = "test"
output_folder = "output_yolo"  # Папка для сохранения результатов
os.makedirs(output_folder, exist_ok=True)
predictions = []
for file_name in tqdm(os.listdir(test_folder)):
    image_path = os.path.join(test_folder, file_name)
    image = Image.open(image_path)
    predicts = predict_yolo_image(image)

    if predicts.xyxy is None:
        continue
    pred_classes = predicts.class_id.tolist()  # Классы объектов
    pred_boxes = predicts.xyxy.tolist()#instances.pred_boxes.tensor.tolist()  # Координаты bbox
    pred_scores = predicts.confidence.tolist()  # Скоринг предсказаний
        
    for i in range(len(pred_boxes)):
        mask , scores = predict_sam(image,pred_boxes[i])
        if float(scores[0]) <= 0.5:
            continue
        rle = pycocotools.mask.encode(np.asfortranarray(mask.astype(np.uint8)[0]))
        predictions.append({
            "image_name": file_name,
            "category_id": 0,  # ID категории
            "bbox": pred_boxes[i],  # Координаты bounding box
            "score": float(scores[0] * 0.5 + pred_scores[i] * 0.5),  # Оценка уверенности
            "segmentation": {
                "size" : rle["size"],
                "counts" : str(rle["counts"])
            }  # Сегментация в формате RLE
        })


# Сохранение предсказаний в JSON
predictions_file = os.path.join(output_folder, "submission.json")
with open(predictions_file, "w") as f:
    json.dump(predictions, f, indent=4)

print(f"Обработка завершена. Результаты сохранены в папке: {output_folder}")